In [1]:
# This notebook accesses the L1 products and runs the C2RCC operator on it to produce L2 products
#

In [1]:
import os
import json
import zipfile
import numpy as np
from PIL import Image
from datetime import date
import matplotlib.pyplot as plt
import xarray as xr
from pathlib import Path
from datetime import date, timedelta
import shutil

from snappy import ProductIO, GPF, HashMap, ProductUtils, PixelPos, GeoPos, ProductData, jpy
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt, make_path_filter

from datetime import datetime
from dateutil.relativedelta import relativedelta
import calendar

%matplotlib inline

INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
Currently installed 9.0.0, available is 10.0.0.
Please visit http://step.esa.int



In [2]:
def list_days_in_month(year, month_text) -> list:
    # Returns a list of integers, the days included in a specific month of a specific year
    
    int_year = int(year)
    # Convert month text to month number
    month_number = datetime.strptime(month_text, "%B").month
    
    # Get the first day of the month
    first_day_of_month = datetime(int_year, month_number, 1)
    
    # Get the number of days in the month
    days_in_month = calendar.monthrange(int_year, month_number)[1]
    
    # Generate the list of days
    days = [first_day_of_month + relativedelta(days=i) for i in range(days_in_month)]
    
    # Extract day numbers
    day_numbers = [day.day for day in days]
    
    return day_numbers

In [3]:
# Input

S3_ROOT_FOLDER = Path('/home/jovyan/earth-observations/notebooks')
ROI_FOLDER = S3_ROOT_FOLDER / 'ROI'

ALGORITHM = 'C2RCC'
ALGORITHM_CONF_FOLDER = S3_ROOT_FOLDER / (ALGORITHM + '_confs')
ALGORITHM_CONF_FILE = ALGORITHM_CONF_FOLDER / 'conf_x1.json'

ROI = 'indre-oslofjord'
YEAR = '2024'
MONTH = 'March'

footprint = geojson_to_wkt(read_geojson(ROI_FOLDER / (ROI + '.geojson')))

INPUT_FOLDER = S3_ROOT_FOLDER / 'S3' / 'L1' / ROI / YEAR / MONTH
OUTPUT_FOLDER = S3_ROOT_FOLDER / 'S3' / 'L2' / ROI / YEAR / MONTH

In [4]:
c2rcc_conf = {
    'validPixelExpression': '!quality_flags.invalid && (!quality_flags.land || quality_flags.fresh_inland_water)',
    'temperature': 15.0,
    'salinity': 35.0,
    'ozone': 330.0,
    'press': 1000.0,
    # 'TSMfakBpart': 1.72,
    # 'TSMfakBwit': 3.1,
    'CHLexp': 1.04,
    'CHLfak': 21.0,
    'thresholdRtosaOOS': 0.005,
    'thresholdAcReflecOos': 0.1,
    'thresholdCloudTDown865': 0.955,
    'outputAsRrs': True,
    'deriveRwFromPathAndTransmittance': True,
    'useEcmwfAuxData': True,
    'outputRtoa': True,
    'outputRtosaGc': True,
    'outputRtosaGcAann': True,
    'outputRpath': True,
    'outputTdown': True,
    'outputTup': True,
    'outputAcReflectance': True,
    'outputRhown': True,
    'outputOos': True,
    'outputKd': True,
    'outputUncertainties': True
}

In [5]:
def start_processing_L1(filename, day, platform):
    
    PROCESSED_OUTPUT_FOLDER = OUTPUT_FOLDER / day / ALGORITHM / platform
    
    # Read the L1 product
    reader = ProductIO.getProductReader('SENTINEL-3')
    product = ProductIO.readProduct(str(filename))

    # Subset the product to the region of interest
    HashMap = jpy.get_type('java.util.HashMap')
    parameters = HashMap()
    
    parameters.put('copyMetadata', True)
    parameters.put('geoRegion', footprint)

    subset_product = GPF.createProduct('Subset', parameters, product)
    
    result = run_c2rcc(subset_product)

    ProductIO.writeProduct(result, f'{PROCESSED_OUTPUT_FOLDER} / {platform}', 'BEAM-DIMAP')  # NetCDF4-CF, BEAM-DIMAP, GeoTIFF, GeoTIFF-BigTIFF
    
    
    # Specify the directory and filename

    # json_filename = 'used_c2rcc_conf.json'
    # file_path = os.path.join(PROCESSED_OUTPUT_FOLDER, json_filename)

    # Ensure the directory exists
    # os.makedirs(PROCESSED_OUTPUT_FOLDER, exist_ok=True)
    
    # Write the dictionary to a JSON file
    # with open(file_path, 'w') as json_file:
    #     json.dump(c2rcc_conf, json_file, indent=4)
    
    # Copy the configuration file to the output folder
    # shutil.copy(ALGORITHM_CONF_FILE, f'{OUTPUT_FOLDER}/{day}/used_c2rcc_conf.json')


In [6]:
def run_c2rcc(product):
    """
    Run C2RCC on the product 
    """

    HashMap = jpy.get_type('java.util.HashMap') 
    parameters = HashMap()
    
    for key, value in c2rcc_conf.items():
        parameters.put(key, value)

    # opSpi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi("c2rcc.olci")
    
    return GPF.createProduct('c2rcc.olci', parameters, product)

In [7]:
for day in list_days_in_month(YEAR, MONTH):
    
    current_source_folder = INPUT_FOLDER / str(day)

    if current_source_folder.exists() and current_source_folder.is_dir():
        
        current_destination_folder = OUTPUT_FOLDER / str(day) / ALGORITHM 
        
        if current_destination_folder.exists() and current_destination_folder.is_dir():
            # L2 product was already created. Skip
            print("L2 product already exist at destination. Skipping")
            continue
        
        try:
            S3A_folder = [f for f in current_source_folder.iterdir() if f.is_dir() and f.name.startswith("S3A")][0]
            S3B_folder = [f for f in current_source_folder.iterdir() if f.is_dir() and f.name.startswith("S3B")][0]
        
            start_processing_L1(S3A_folder, str(day), "S3A")
            start_processing_L1(S3B_folder, str(day), "S3B")
            
        except IndexError:
            print("Less than the two expected products")
        
    else:
        print(f"S3 L1 product for {day} of {MONTH} {YEAR} not found. Skipping")

S3 L1 product for 1 of March 2024 not found. Skipping
S3 L1 product for 2 of March 2024 not found. Skipping
S3 L1 product for 3 of March 2024 not found. Skipping
S3 L1 product for 4 of March 2024 not found. Skipping
S3 L1 product for 5 of March 2024 not found. Skipping
S3 L1 product for 6 of March 2024 not found. Skipping
S3 L1 product for 7 of March 2024 not found. Skipping
S3 L1 product for 8 of March 2024 not found. Skipping
S3 L1 product for 9 of March 2024 not found. Skipping
S3 L1 product for 10 of March 2024 not found. Skipping
S3 L1 product for 11 of March 2024 not found. Skipping
S3 L1 product for 12 of March 2024 not found. Skipping
S3 L1 product for 13 of March 2024 not found. Skipping
S3 L1 product for 14 of March 2024 not found. Skipping
S3 L1 product for 15 of March 2024 not found. Skipping
S3 L1 product for 16 of March 2024 not found. Skipping
S3 L1 product for 17 of March 2024 not found. Skipping
S3 L1 product for 18 of March 2024 not found. Skipping
S3 L1 product for 1

INFO: org.hsqldb.persist.Logger: dataFileCache open start
INFO: org.esa.s3tbx.c2rcc.olci.C2rccOlciOperator: c2rcc initial tile : null, configured tile: java.awt.Dimension[width=1217,height=1023]



100% done.
Preparing computation
50%100% done.

100% done.
Preparing computation


INFO: org.esa.s3tbx.c2rcc.olci.C2rccOlciOperator: c2rcc initial tile : null, configured tile: java.awt.Dimension[width=1217,height=1023]


50%100% done.

100% done.
Preparing computation
50%100% done.

100% done.


INFO: org.esa.s3tbx.c2rcc.olci.C2rccOlciOperator: c2rcc initial tile : null, configured tile: java.awt.Dimension[width=1217,height=1023]


Preparing computation
50%100% done.
S3 L1 product for 24 of March 2024 not found. Skipping
S3 L1 product for 25 of March 2024 not found. Skipping
S3 L1 product for 26 of March 2024 not found. Skipping
S3 L1 product for 27 of March 2024 not found. Skipping
S3 L1 product for 28 of March 2024 not found. Skipping
S3 L1 product for 29 of March 2024 not found. Skipping
S3 L1 product for 30 of March 2024 not found. Skipping
S3 L1 product for 31 of March 2024 not found. Skipping


INFO: org.esa.s3tbx.c2rcc.olci.C2rccOlciOperator: c2rcc initial tile : null, configured tile: java.awt.Dimension[width=1217,height=1023]


In [9]:
opSpi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi("c2rcc.olci")

paramDescList = opSpi.getOperatorDescriptor().getParameterDescriptors()
for param in paramDescList:
    print(param.getName())

validPixelExpression
salinity
temperature
ozone
press
TSMfakBpart
TSMfakBwit
CHLexp
CHLfak
thresholdRtosaOOS
thresholdAcReflecOos
thresholdCloudTDown865
atmosphericAuxDataPath
alternativeNNPath
outputAsRrs
deriveRwFromPathAndTransmittance
useEcmwfAuxData
outputRtoa
outputRtosaGc
outputRtosaGcAann
outputRpath
outputTdown
outputTup
outputAcReflectance
outputRhown
outputOos
outputKd
outputUncertainties
